In [1]:
import numpy as np
import sympy as sym
import pydae.build_cffi as db

In [32]:
def dynph_conv(var1,var2,k_dict,k,verb = False):
    k_list = list(k_dict.keys())
    conv = 0
    conj_var2 = False
    if 'conj' in var2:
        var2 = var2.split('(')[1][:-1]
        conj_var2 = True
    conv = 0
    for l in range(-10,10):
        m = k - l
        #print(f'm = {m:3d}, l= {l:3d}')

        if m in k_list  or -m in k_list:
            if l in k_list or -l in k_list:
                sym1r,sym1i = sym.symbols(f'{var1}{k_dict[np.abs(m)]}r,{var1}{k_dict[np.abs(m)]}i', real=True)
                sym2r,sym2i = sym.symbols(f'{var2}{k_dict[np.abs(l)]}r,{var2}{k_dict[np.abs(l)]}i', real=True)
                sym_1 = sym1r + sym.I*sym1i
                sym_2 = sym2r + sym.I*sym2i
                
                if conj_var2:
                    
                    if m < 0: 
                        if verb: print(f'conj(x_({-m:2d})) * y_({l:2d})')
                        conv += sym_1*sym.conjugate(sym_2) 
                    elif l < 0:
                        if verb: print(f'x_({m:2d}) * conj(y_({-l:2d}))')
                        conv += sym.conjugate(sym_1)*sym_2
                    else:
                        if verb: print(f'x_({m:2d})) * y_({l:2d})')
                        conv += sym_1*sym_2                    
                    
                else:
                    sym_2 = (sym2r + sym.I*sym2i)
                        
                    if m < 0: 
                        if verb: print(f'conj(x_({-m:2d})) * y_({l:2d})')
                        conv += sym.conjugate(sym_1)*sym_2
                    elif l < 0:
                        if verb: print(f'x_({m:2d}) * conj(y_({-l:2d}))')
                        conv += sym_1*sym.conjugate(sym_2)
                    else:
                        if verb: print(f'x_({m:2d})) * y_({l:2d})')
                        conv += sym_1*sym_2
    return conv

k_dict = {-1:'n',0:0,1:'p'}
#k_dict = {'n':-1,'p':1}
#k_dict = {'n':-1,'0':0,'p':1,'2':2}
xy = dynph_conv('v','i',k_dict,0)
xy

(I*i0i + i0r)*(I*v0i + v0r) + (-I*ipi + ipr)*(I*vpi + vpr) + (I*ipi + ipr)*(-I*vpi + vpr)

In [33]:
# https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=4538460&tag=1

R,L,omega = sym.symbols('R,L,omega', real=True) 

v_tdnr,v_tdni,v_tqnr,v_tqni = sym.symbols('v_tdnr,v_tdni,v_tqnr,v_tqni', real=True)
v_td0r,v_td0i,v_tq0r,v_tq0i = sym.symbols('v_td0r,v_td0i,v_tq0r,v_tq0i', real=True)
v_tdpr,v_tdpi,v_tqpr,v_tqpi = sym.symbols('v_tdpr,v_tdpi,v_tqpr,v_tqpi', real=True)

i_dnr,i_dni,i_qnr,i_qni = sym.symbols('i_dnr,i_dni,i_qnr,i_qni', real=True)
i_d0r,i_d0i,i_q0r,i_q0i = sym.symbols('i_d0r,i_d0i,i_q0r,i_q0i', real=True)
i_dpr,i_dpi,i_qpr,i_qpi = sym.symbols('i_dpr,i_dpi,i_qpr,i_qpi', real=True)

v_sdnr,v_sdni,v_sqnr,v_sqni = sym.symbols('v_sdnr,v_sdni,v_sqnr,v_sqni', real=True)
v_sd0r,v_sd0i,v_sq0r,v_sq0i = sym.symbols('v_sd0r,v_sd0i,v_sq0r,v_sq0i', real=True)
v_sdpr,v_sdpi,v_sqpr,v_sqpi = sym.symbols('v_sdpr,v_sdpi,v_sqpr,v_sqpi', real=True)


j = sym.I

v_tdn = v_tdnr + j*v_tdni
v_tqn = v_tqnr + j*v_tqni
v_td0 = v_td0r + j*v_td0i
v_tq0 = v_tq0r + j*v_tq0i
v_tdp = v_tdpr + j*v_tdpi
v_tqp = v_tqpr + j*v_tqpi


i_dn = i_dnr + j*i_dni
i_qn = i_qnr + j*i_qni
i_d0 = i_d0r + j*i_d0i
i_q0 = i_q0r + j*i_q0i
i_dp = i_dpr + j*i_dpi
i_qp = i_qpr + j*i_qpi


v_sdn = v_sdnr + j*v_sdni
v_sqn = v_sqnr + j*v_sqni
v_sd0 = v_sd0r + j*v_sd0i
v_sq0 = v_sq0r + j*v_sq0i
v_sdp = v_sdpr + j*v_sdpi
v_sqp = v_sqpr + j*v_sqpi


# di_d = (v_d - R * i_d)/L
# di_q = (v_q - R * i_q)/L

#K = -2


#K = -1
di_dn = 1/L*(v_tdn - R * i_dn - v_sdn) - j*(-1)*omega*i_dn
di_qn = 1/L*(v_tqn - R * i_qn - v_sqn) - j*(-1)*omega*i_qn

#K = 0
di_d0 = 1/L*(v_td0 - R * i_d0 - v_sd0) - j*( 0)*omega*i_d0
di_q0 = 1/L*(v_tq0 - R * i_q0 - v_sq0) - j*( 0)*omega*i_q0

#K = 1
di_dp = 1/L*(v_tdp - R * i_dp - v_sdp) - j*(+1)*omega*i_dp
di_qp = 1/L*(v_tqp - R * i_qp - v_sqp) - j*(+1)*omega*i_qp

k_dict = {-1:'n',0:0,1:'p'}
s_abc_tn = 1.5*(dynph_conv('v_td','conj(i_d)',k_dict,-1) + dynph_conv('v_tq','conj(i_q)',k_dict,-1))
s_abc_t0 = 1.5*(dynph_conv('v_td','conj(i_d)',k_dict, 0) + dynph_conv('v_tq','conj(i_q)',k_dict, 0))
s_abc_tp = 1.5*(dynph_conv('v_td','conj(i_d)',k_dict, 1) + dynph_conv('v_tq','conj(i_q)',k_dict, 1))
s_abc_t2 = 1.5*(dynph_conv('v_td','conj(i_d)',k_dict, 2) + dynph_conv('v_tq','conj(i_q)',k_dict, 2))

s_t_abc = s_abc_tn + s_abc_t0 + s_abc_tp #+ s_abc_t2

#i_n = i_dn + j*i_qn
#i_0 = i_d0 + j*i_q0
#i_p = i_dp + j*i_qp
#i_2 = i_d2 + j*i_q2
#
#v_n = v_dn + j*v_qn
#v_0 = v_d0 + j*v_q0
#v_p = v_dp + j*v_qp
#v_2 = v_d2 + j*v_q2

#s_avg = v_n*conj(i_n) + v_0*conj(i_0) +v_p*conj(i_p) + v_2*conj(i_2)
#s_osc_n1 = v_n*conj(i_0) + v_0*conj(i_p) + v_p*conj(i_2)
#s_osc_n2 = v_n*conj(i_0) + v_0*conj(i_p) + v_p*conj(i_2)
#s_osc_n = v_n*conj(i_0) + v_0*conj(i_p) + v_p*conj(i_2) 
#s_osc_0 = v_0*conj(i_n) + v_0*conj(i_p) +v_0*conj(i_2) 
#v_t = v_n*exp(-jwt)

s_abc_sn = 1.5*(dynph_conv('v_sd','conj(i_d)',k_dict,-1) + dynph_conv('v_sq','conj(i_q)',k_dict,-1))
s_abc_s0 = 1.5*(dynph_conv('v_sd','conj(i_d)',k_dict, 0) + dynph_conv('v_sq','conj(i_q)',k_dict, 0))
s_abc_sp = 1.5*(dynph_conv('v_sd','conj(i_d)',k_dict, 1) + dynph_conv('v_sq','conj(i_q)',k_dict, 1))
s_abc_s2 = 1.5*(dynph_conv('v_sd','conj(i_d)',k_dict, 2) + dynph_conv('v_sq','conj(i_q)',k_dict, 2))

s_s_abc = s_abc_sn + s_abc_s0 + s_abc_sp + s_abc_s2

params_dict = {'R':0.1,'L':0.01}


f_list = [sym.re(di_dn),sym.im(di_dn),
          sym.re(di_qn),sym.im(di_qn),
          sym.re(di_d0),sym.im(di_d0),
          sym.re(di_q0),sym.im(di_q0),
          sym.re(di_dp),sym.im(di_dp),
          sym.re(di_qp),sym.im(di_qp)]
x_list = [i_dnr,i_dni,
          i_qnr,i_qni,
          i_d0r,i_d0i,
          i_q0r,i_q0i,
          i_dpr,i_dpi,
          i_qpr,i_qpi,
         ]
g_list = []
y_list = []

u_ini_dict = {'omega':2*np.pi*50,
'v_tdnr':0.0,'v_tqnr':0.0,
'v_td0r':0.0,'v_tq0r':0.0,
'v_tdpr':0.0,'v_tqpr':0.0,
'v_tdni':0.0,'v_tqni':0.0,
'v_td0i':0.0,'v_tq0i':0.0,
'v_tdpi':0.0,'v_tqpi':0.0,
'v_sdnr':0.0,'v_sqnr':0.0,
'v_sdpr':0.0,'v_sqpr':0.0,
'v_sd0r':0.0,'v_sq0r':0.0,
'v_sdni':0.0,'v_sqni':0.0,
'v_sd0i':0.0,'v_sq0i':0.0,
'v_sdpi':0.0,'v_sqpi':0.0,
}
u_run_dict = u_ini_dict


h_dict = {
'v_tdnr':v_tdnr,'v_tqnr':v_tqnr,
'v_td0r':v_td0r,'v_tq0r':v_tq0r,
'v_tdpr':v_tdpr,'v_tqpr':v_tqpr,
'v_tdni':v_tdni,'v_tqni':v_tqni,
'v_td0i':v_td0i,'v_tq0i':v_tq0i,
'v_tdpi':v_tdpi,'v_tqpi':v_tqpi,
'v_sdnr':v_sdnr,'v_sqnr':v_sqnr,
'v_sd0r':v_sd0r,'v_sq0r':v_sq0r,
'v_sdpr':v_sdpr,'v_sqpr':v_sqpr,
'v_sdni':v_sdni,'v_sqni':v_sqni,
'v_sd0i':v_sd0i,'v_sq0i':v_sq0i,
'v_sdpi':v_sdpi,'v_sqpi':v_sqpi,
's_abc_tnr':sym.re(s_abc_tn),'s_abc_tni':sym.im(s_abc_tn),
's_abc_t0r':sym.re(s_abc_t0),'s_abc_t0i':sym.im(s_abc_t0),
's_abc_tpr':sym.re(s_abc_tp),'s_abc_tpi':sym.im(s_abc_tp),
's_abc_t2r':sym.re(s_abc_t2),'s_abc_t2i':sym.im(s_abc_t2),
's_abc_snr':sym.re(s_abc_sn),'s_abc_sni':sym.im(s_abc_sn),
's_abc_s0r':sym.re(s_abc_s0),'s_abc_s0i':sym.im(s_abc_s0),
's_abc_spr':sym.re(s_abc_sp),'s_abc_spi':sym.im(s_abc_sp),
's_abc_s2r':sym.re(s_abc_s2),'s_abc_s2i':sym.im(s_abc_s2),
}

In [40]:
s_abc_s0

1.5*(I*i_d0i + i_d0r)*(I*v_sd0i + v_sd0r) + 1.5*(-I*i_dpi + i_dpr)*(I*v_sdpi + v_sdpr) + 1.5*(I*i_dpi + i_dpr)*(-I*v_sdpi + v_sdpr) + 1.5*(I*i_q0i + i_q0r)*(I*v_sq0i + v_sq0r) + 1.5*(-I*i_qpi + i_qpr)*(I*v_sqpi + v_sqpr) + 1.5*(I*i_qpi + i_qpr)*(-I*v_sqpi + v_sqpr)

In [38]:
sys = {'name':f"rl_dynph2",
       'params_dict':params_dict,
       'f_list':f_list,
       'g_list':g_list,
       'x_list':x_list,
       'y_ini_list':y_list,
       'y_run_list':y_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h_dict':h_dict
      }

bldr = db.builder(sys)
bldr.build()

system without algebraic equations, adding dummy algebraic equation
Compilation time: 8.12 s


In [12]:
dynph_conv('v_sd','i_d',k_dict, 0)

(I*i_d0i + i_d0r)*(I*v_sd0i + v_sd0r) + (-I*i_dpi + i_dpr)*(I*v_sdpi + v_sdpr) + (I*i_dpi + i_dpr)*(-I*v_sdpi + v_sdpr)

In [7]:
print(bldr.source)

void f_ini_eval(double *out,double *x,double *y,double *u,double *p,double Dt){

out[0] = -x[1]*u[0] + (-p[0]*x[0] - u[13] + u[1])/p[1];
out[1] = x[0]*u[0] + (-p[0]*x[1] - u[17] + u[7])/p[1];
out[2] = -x[3]*u[0] + (-p[0]*x[2] - u[14] + u[2])/p[1];
out[3] = x[2]*u[0] + (-p[0]*x[3] - u[18] + u[8])/p[1];
out[4] = (-p[0]*x[4] - u[15] + u[3])/p[1];
out[5] = (-p[0]*x[5] - u[19] + u[9])/p[1];
out[6] = (-p[0]*x[6] - u[16] + u[4])/p[1];
out[7] = (-p[0]*x[7] - u[20] + u[10])/p[1];
out[8] = x[9]*u[0] + (-p[0]*x[8] - v_sdpr + u[5])/p[1];
out[9] = -x[8]*u[0] + (-p[0]*x[9] - u[21] + u[11])/p[1];
out[10] = x[11]*u[0] + (-p[0]*x[10] - v_sqpr + u[6])/p[1];
out[11] = -x[10]*u[0] + (-p[0]*x[11] - u[22] + u[12])/p[1];

}
void g_ini_eval(double *out,double *x,double *y,double *u,double *p,double Dt){

out[0] = u[23] - y[0];

}
void f_run_eval(double *out,double *x,double *y,double *u,double *p,double Dt){

out[0] = -x[1]*u[0] + (-p[0]*x[0] - u[13] + u[1])/p[1];
out[1] = x[0]*u[0] + (-p[0]*x[1] - u[17] + u[

In [6]:
V_abc = np.array([238,195,217])
np.max(V_abc)/np.sum(V_abc)/3

0.12205128205128206

In [30]:
params_dict = {'R':0.1,'L':0.01}
u_ini_dict = {'v_d0r':0.0,'v_q0r':0.0,
              'v_d2r':0.0,'v_q2r':0.0,
              'v_d0i':0.0,'v_q0i':0.0,
              'v_d2i':0.0,'v_q2i':0.0,
              'omega':2*np.pi*50}   

u_run_dict = u_ini_dict

i_d0r,i_q0r,i_d2r,i_q2r,i_d0i,i_q0i,i_d2i,i_q2i = sym.symbols('i_d0r,i_q0r,i_d2r,i_q2r,i_d0i,i_q0i,i_d2i,i_q2i', real=True)
v_d0r,v_q0r,v_d2r,v_q2r,v_d0i,v_q0i,v_d2i,v_q2i = sym.symbols('v_d0r,v_q0r,v_d2r,v_q2r,v_d0i,v_q0i,v_d2i,v_q2i', real=True)
R,L,omega = sym.symbols('R,L,omega', real=True) 

v_d0 = v_d0r + sym.I*v_d0i
v_q0 = v_q0r + sym.I*v_q0i
v_d2 = v_d2r + sym.I*v_d2i
v_q2 = v_q2r + sym.I*v_q2i

i_d0 = i_d0r + sym.I*i_d0i
i_q0 = i_q0r + sym.I*i_q0i
i_d2 = i_d2r + sym.I*i_d2i
i_q2 = i_q2r + sym.I*i_q2i

K = 0
di_d0 = (v_d0 - R * i_d0 + L*omega*i_q0 - sym.I*K*omega*L*i_d0)/L
di_q0 = (v_q0 - R * i_q0 - L*omega*i_d0 - sym.I*K*omega*L*i_q0)/L

di_d0r = sym.re(di_d0)
di_q0r = sym.re(di_q0)
di_d0i = sym.im(di_d0)
di_q0i = sym.im(di_q0)

i_q2_ = -sym.I*i_d2 
i_d2_ =  sym.I*i_q2 


K = 2
di_d2 = (v_d2 - R * i_d2 + L*omega*i_q2 - sym.I*K*omega*L*i_d2)/L
di_q2 = (v_q2 - R * i_q2 - L*omega*i_d2 - sym.I*K*omega*L*i_q2)/L

di_d2r = sym.re(di_d2)
di_q2r = sym.re(di_q2)
di_d2i = sym.im(di_d2)
di_q2i = sym.im(di_q2)

f_list = [di_d0r,di_q0r,di_d0i,di_q0i,di_d2r,di_q2r,di_d2i,di_q2i]
x_list = [ i_d0r, i_q0r, i_d0i, i_q0i, i_d2r, i_q2r, i_d2i, i_q2i]
g_list = []
y_list = []

h_dict = {'v_d0r':v_d0r,'v_q0r':v_q0r}

In [12]:
f_list

[-omega*(re(i_dni) + im(i_dnr)) + (-R*(re(i_dnr) - im(i_dni)) + re(v_dnr) - im(v_dni))/L,
 omega*(re(i_qnr) - im(i_qni)) + (-R*(re(i_qni) + im(i_qnr)) + re(v_qni) + im(v_qnr))/L,
 (-R*(re(i_d0r) - im(i_d0i)) + re(v_d0r) - im(v_d0i))/L,
 (-R*(re(i_q0i) + im(i_q0r)) + re(v_q0i) + im(v_q0r))/L,
 omega*(re(i_dpi) + im(i_dpr)) + (-R*(re(i_dpr) - im(i_dpi)) + re(v_dpr) - im(v_dpi))/L,
 -omega*(re(i_qpr) - im(i_qpi)) + (-R*(re(i_qpi) + im(i_qpr)) + re(v_qpi) + im(v_qpr))/L]

In [31]:
sys = {'name':f"rl_dp",
       'params_dict':params_dict,
       'f_list':f_list,
       'g_list':g_list,
       'x_list':x_list,
       'y_ini_list':y_list,
       'y_run_list':y_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h_dict':h_dict
      }

bldr = db.builder(sys)
bldr.build()

system without algebraic equations, adding dummy algebraic equation
Compilation time: 2.90 s


In [34]:
f_list

[(L*i_q0r*omega - R*i_d0r + v_d0r)/L,
 (-L*i_d0r*omega - R*i_q0r + v_q0r)/L,
 (L*i_q0i*omega - R*i_d0i + v_d0i)/L,
 (-L*i_d0i*omega - R*i_q0i + v_q0i)/L,
 (2*L*i_d2i*omega + L*i_q2r*omega - R*i_d2r + v_d2r)/L,
 (-L*i_d2r*omega + 2*L*i_q2i*omega - R*i_q2r + v_q2r)/L,
 (-2*L*i_d2r*omega + L*i_q2i*omega - R*i_d2i + v_d2i)/L,
 (-L*i_d2i*omega - 2*L*i_q2r*omega - R*i_q2i + v_q2i)/L]

In [33]:
x_list

[i_d0r, i_q0r, i_d0i, i_q0i, i_d2r, i_q2r, i_d2i, i_q2i]